<a href="https://colab.research.google.com/github/giljae/sandbox/blob/master/LLM/sample_text_sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 감정 분석 (Sentiment Analysis):

Text Classification(텍스트 분류)은 주어진 텍스트(문장이나 문서)를 미리 정해진 카테고리(범주) 중 하나로 분류하는 작업을 의미합니다.

가장 대표적인 예시가 바로 감성 분석(Sentiment Analysis)입니다.

샘플 코드를 실행하기에 앞서 transfomers를 설치해야 합니다.

In [ ]:
# huggingface transformers 설치
!pip install transformers

## Copycats/koelectra-base-v3-generalized-sentiment-analysis 모델 테스트

In [ ]:
# import library
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# load model
tokenizer = AutoTokenizer.from_pretrained("Copycats/koelectra-base-v3-generalized-sentiment-analysis")
model = AutoModelForSequenceClassification.from_pretrained("Copycats/koelectra-base-v3-generalized-sentiment-analysis")
sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

# target reviews
review_list = [
    "음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.",
    "분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.",
    "기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.",
    "배우들 연기는 정말 최고였어요.",
    '한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?',
    "오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.",
    "영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다."
]

# predict
for idx, review in enumerate(review_list):
  pred = sentiment_classifier(review)
  print(f'{review}\n>> {pred[0]}')


결과:
- label 0 : 부정적인 리뷰
- label 1 : 긍정적인 리뷰

수행 결과는 아래처럼 나옵니다.
```
음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.
>> {'label': '0', 'score': 0.9508960247039795}

분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.
>> {'label': '0', 'score': 0.6903115510940552}

기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.
>> {'label': '0', 'score': 0.9847258925437927}

배우들 연기는 정말 최고였어요.
>> {'label': '1', 'score': 0.9640251994132996}

한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?
>> {'label': '0', 'score': 0.9988909363746643}

오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.
>> {'label': '1', 'score': 0.9475674629211426}

영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다.
>> {'label': '0', 'score': 0.9037595987319946}
```

테스트 문장을 긍정과 부정이 섞인 복합감정인 것으로 테스트를 하였습니다. 결과는 위의 수치와 같습니다. 조금더 자세히 살펴보면,

### 문장의 구조 (A-but-B)
"A-but-B" ("A(긍정)이지만 B(부정)이다") 구조에서, 사람들은 보통 B에 더 강한 강조점이나 최종 결론을 둡니다.
- A(긍정): 음식 맛은 정말 훌륭한데
- B(부정): 가게가 너무 시끄럽고 직원이 불친절해요.

이 문장은 "맛은 좋았으나, 시끄럽고 불친절해서 (전반적으로) 별로였다"라는 뉘앙스를 강하게 풍깁니다.

### 부정적인 요소의 '양'
단순히 키워드만 보더라도,
- 긍정 키워드: 훌륭한데 (1개)
- 부정 키워드: 시끄럽고, 불친절해요 (2개)

일반적인 감성 분석 모델은 문장 전체를 보고 하나의 레이블(긍정/부정)을 결정해야 할 때, 부정적인 요소가 2가지나 명확하게 언급되었기 때문에 부정으로 판단할 가능성이 큽니다.

### 감정 분석 모델의 한계와 종류
문장을 어떻게 분석하는지는 사실 모델의 수준에 따라 다릅니다.

#### 일반적인 감성 분석 (Sentence-level):
대부분의 표준 모델은 문장 전체의 '종합적인' 분위기를 판단하도록 학습됩니다.

이 경우, 긍정적인 면(맛)이 언급되었음에도 불구하고 부정적인 면(소음, 불친절)이 경험을 망쳤다고 해석하여 '부정' (또는 별점 1~2점)으로 분류할 것입니다.

#### 속성 기반 감성 분석 (ABSA - Aspect-Based Sentiment Analysis):
더 발전된 형태의 분석입니다. 문장을 '주제'별로 쪼개서 각각의 감성을 분석합니다.
만약 이 문장을 ABSA 모델로 분석한다면 이런 결과가 나옵니다.

- 음식 맛: 긍정(Positive)
- 가게 분위기/소음: 부정(Negative)
- 직원/서비스: 부정(Negative)

## yangheng/deberta-v3-base-absa-v1.1 모델 테스트

In [ ]:
# import library
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TextClassificationPipeline

# load model
#tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")
#model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

tokenizer = AutoTokenizer.from_pretrained("team-lucid/deberta-v3-base-korean")
model = AutoModelForSequenceClassification.from_pretrained("team-lucid/deberta-v3-base-korean")

sentiment_classifier = TextClassificationPipeline(tokenizer=tokenizer, model=model)

# target reviews
review_list = [
    "음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.",
    "분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.",
    "기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.",
    "배우들 연기는 정말 최고였어요.",
    "여기 나빠요.",
    '한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?',
    "오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.",
    "영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다."
]

# predict
for idx, review in enumerate(review_list):
  pred = sentiment_classifier(review)
  print(f'{review}\n>> {pred[0]}')


Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at team-lucid/deberta-v3-base-korean and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Device set to use cpu


음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.
>> {'label': 'LABEL_1', 'score': 0.5080758333206177}
분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.
>> {'label': 'LABEL_1', 'score': 0.5078545212745667}
기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.
>> {'label': 'LABEL_1', 'score': 0.507517397403717}
배우들 연기는 정말 최고였어요.
>> {'label': 'LABEL_1', 'score': 0.5079139471054077}
여기 나빠요.
>> {'label': 'LABEL_1', 'score': 0.5088595747947693}
한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?
>> {'label': 'LABEL_1', 'score': 0.5073701739311218}
오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.
>> {'label': 'LABEL_1', 'score': 0.5097130537033081}
영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다.
>> {'label': 'LABEL_1', 'score': 0.508996844291687}


결과는 아래와 같이 나옵니다.
```
음식 맛은 정말 훌륭한데, 가게가 너무 시끄럽고 직원이 불친절해요.
>> {'label': 'Positive', 'score': 0.7525179386138916}

분위기나 인테리어는 정말 예쁜데, 가격에 비해 양이 너무 적었어요.
>> {'label': 'Positive', 'score': 0.5790202617645264}

기능은 다양한데, 사용법이 너무 복잡해서 오히려 불편해요.
>> {'label': 'Negative', 'score': 0.5635974407196045}

배우들 연기는 정말 최고였어요.
>> {'label': 'Positive', 'score': 0.9434264302253723}

한번입었는데 옆에 봉제선 다 풀리고 실밥도 계속 나옵니다. 마감 처리 너무 엉망 아닌가요?
>> {'label': 'Negative', 'score': 0.9397663474082947}

오랜만에 친구들을 만나서 반가웠지만, 헤어질 때는 너무 아쉬웠어요.
>> {'label': 'Positive', 'score': 0.8371114134788513}

영상미는 화려하고 좋았지만, 결말이 너무 허무했습니다.
>> {'label': 'Positive', 'score': 0.862045168876648}
```

# 결론


## Copycats/koelectra-base-v3-generalized-sentiment-analysis 모델
이 모델이 복합 감정 문장을 훨씬 더 잘 이해한다고 생각합니다.
- 영상미는... 좋았지만, 결말이... 허무했습니다. -> Negative (정확)
- 음식 맛은... 훌륭한데, 직원이... 불친절해요. -> Negative (정확)
- 분위기나... 예쁜데, 양이... 적었어요. -> Negative (정확)
- 기능은... 다양한데, 사용법이... 불편해요. -> Negative (정확)

문장 앞부분의 긍정적인 칭찬에도 불구하고, 최종적으로 전달하려는 의도(불편함, 불친절함, 양 적음, 허무함)가 부정적이라는 것을 정확하게 포착했다고 생각합니다.

## yangheng/deberta-v3-base-absa-v1.1 모델
이 모델은 "A이지만 B이다." 구조의 문맥을 잘 파악하지 못하는 것 같습니다.
- 영상미는... 좋았지만, 결말이... 허무했습니다. -> Positive (부정확)
- 음식 맛은... 훌륭한데, 직원이... 불친절해요. -> Positive (부정확)
- 분위기나... 예쁜데, 양이... 적었어요. -> Positive (부정확)

이 모델은 문장 앞부분에 나오는 훌륭한데, 예쁜데, 좋았지만 등과 같은 긍정적인 단어에 너무 큰 가중치를 두어, 뒤에 아노는 부정적인 결론을 놓친다고 생각합니다.